In [1]:
import numpy as np
import pandas as pd
from pyscheduling.FS import FmCmax, FlowShop
import pickle
import time
import matplotlib.pyplot as plt
import numpy as np
import random
import numpy as np

## Makespan

In [2]:
def compute_makespan(schedule, p):
    _, m = p.shape
    n = len(schedule)
    c = [[0]*m for i in range(n)]
    for i in range(n):
        for j in range(m):
            if i == 0 and j == 0:
                c[i][j] = p[schedule[i]][j]
            elif i == 0:
                c[i][j] = c[i][j-1] + p[schedule[i]][j]
            elif j == 0:
                c[i][j] = c[i-1][j] + p[schedule[i]][j]
            else:
                c[i][j] = max(c[i][j-1], c[i-1][j]) + p[schedule[i]][j]
    return c[n-1][m-1]

## Jaya Algorithm

## Brève introduction de l'algorithme de Jaya
L'algorithme de Jaya est basé sur le principe que la solution au problème donné doit se rapprocher vers la meilleure solution connue et s'éloigne de la pire solution.
Les étapes de l'application de l'algorithme de Jaya sont brièvement résumées ci-dessous :
<ul>
<li>Initialiser la taille de la population et le critère de terminaison (max-iter)</li>
<li>Identifier la pire et la meilleure solution dans la population</li>
<li>Modifier la variable de conception (dans notre cas la priorite (ordre) des jobs) des autres solutions sur la base de la meilleure et de la pire solution selon l'équation (1)</li>
<li>Comparez la solution actualisée à la solution précédente. Si la solution actualisée est meilleure, remplacez-la sinon conservez l'ancienne solution.</li>
<li>Déclarer la solution optimale</li>
</ul>
<br/>
<img width="700" height="500" src="images/jaya_formula1.png"/>
<!-- <i>
   <ul> $x'$<sub><i>$i,k,l$</i></sub> = $x$<sub><i>$i,k,l$</i></sub> + $r$<sub><i>$1,i,l$</i></sub> * ( $x$<sub><i>$i,k,lbest$</i></sub> - |$x$<sub><i>$i,k,l$</i></sub>|) + $r$<sub><i>$2,i,l$</i></sub> * ( $x$<sub><i>$i,k,lworst$</i></sub> - |$x$<sub><i>$i,k,l$</i></sub>|) ......................... (1)</ul>
<p>
<br/>
    <ul>where:</ul>
<ul>
  <ol>  $x$<sub><i>$i,k,l$</i></sub> : la valeur d'une i ème variable dans la k ème population au cours de la l ème itération</ol>
  <ol>$x$<sub><i>$i,k,lbest$</i></sub> : la valeur d'une ième variable dans la population ayant la meilleure solution </ol>
  <ol>$x$<sub><i>$i,k,lworst$</i></sub> : la valeur d'une ième variable dans la population ayant la plus mauvaise solution </ol>
  <ol>$x'$<sub><i>$i,k,l$</i></sub> : valeur actualisée de $x$<sub><i>$i,k,l$</i></sub> </ol>
  <ol>$r$<sub><i>$1,i,l$</i></sub> $r$<sub><i>$2,i,l$</i></sub> : des nombres aléatoires pour la ième variable de la lème itération pour la meilleure et la pire solution respectivement dans  [0, 1] </ol>
 </ul>
</p>
   </i> -->


## Implementation

In [23]:
def generate_job_proirity(num_jobs):
    p=[]
    for i in range(num_jobs):
        p.append(1+np.random.random()*(num_jobs-1))
    return p
def proiroty_to_sequence(job_priority):
    s=[]
    p=job_priority.copy()
    for i in range(len(p)):
        s.append(np.argmax(p))
        p[s[-1]]=-float('inf')
    return s
def newP(old,best,worst):
    return old+np.random.random()*(best-np.abs(old))-np.random.random()*(worst-np.abs(old))
def jaya_algo(num_jobs,size_p,process_times,max_iter):
    #generate population
    population=[]
    makespans=[_ for _ in range(size_p)]
    
    #iter 1
    for i in range(size_p):
        population.append(generate_job_proirity(num_jobs))
    #covert to sequence
    for i in range(size_p):
        makespans[i]=compute_makespan(proiroty_to_sequence(population[i]),process_times)
    pi_best=np.min(makespans)
    i_min=np.argmin(makespans)
    i_max=np.argmax(makespans)
    pi_worst=np.max(makespans)
    # other iters
    for _ in range(max_iter-1):
        for i in range(size_p):
            for j in range(num_jobs):
                population[i][j]=newP(population[i][j],population[i_min][j],population[i_max][j])
        for i in range(size_p):
            makespans[i]=compute_makespan(proiroty_to_sequence(population[i]),process_times)
        pi_best=np.min(makespans)
        i_min=np.argmin(makespans)
        pi_worst=np.max(makespans)
        i_max=np.argmax(makespans)
    return proiroty_to_sequence(population[np.argmin(makespans)])

## Tests

In [14]:
p=generate_job_proirity(3)
proiroty_to_sequence(p),p,np.argmax(p)

([2, 0, 1], [1.199738443623204, 1.1693589977092422, 1.997453518727237], 2)

### 1- Instance random

In [29]:
instance=FmCmax.FmCmax_Instance.read_txt("../TP02-Heuristiques/data/random_instance.txt")
n = instance.n
m = instance.m
M = np.array(instance.P)
size_population=250
max_iter=100
schedule=jaya_algo(n,size_population,M,max_iter)
print("makespan",compute_makespan(schedule,M))

makespan 1102


### 2- Instance Taillard

In [31]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=jaya_algo(20,500,M,100)
    print("instance",i+1,":",100*(compute_makespan(schedule,M)-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.9565857247976454 %
instance 3 : 4.25531914893617 %
instance 4 : 4.176334106728539 %
instance 5 : 2.9935275080906147 %
instance 6 : 5.188284518828452 %
instance 7 : 4.600484261501211 %
instance 8 : 3.067993366500829 %
instance 9 : 6.260162601626016 %
instance 10 : 6.046931407942238 %


In [32]:
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=jaya_algo(20,5000,M,100)
    print("instance",i+1,":",100*(compute_makespan(schedule,M)-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.0 %
instance 3 : 2.4051803885291396 %
instance 4 : 1.237432327919567 %
instance 5 : 1.132686084142395 %
instance 6 : 2.426778242677824 %
instance 7 : 1.4527845036319613 %
instance 8 : 1.9900497512437811 %
instance 9 : 2.5203252032520327 %
instance 10 : 4.332129963898917 %


In [215]:
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=jaya_algo(20,10000,M,10)
    print("instance",i+1,":",100*(compute_makespan(schedule,M)-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.515084621044886 %
instance 3 : 3.145235892691952 %
instance 4 : 1.3921113689095128 %
instance 5 : 1.4563106796116505 %
instance 6 : 2.426778242677824 %
instance 7 : 0.9685230024213075 %
instance 8 : 1.492537313432836 %
instance 9 : 1.8699186991869918 %
instance 10 : 2.707581227436823 %


## Genetic algorithm
<!-- link (https://dergipark.org.tr/en/download/article-file/950358) -->

### Utils

In [6]:
#utilities
def init_population(n_jobs,size_p):
    population=[]
    i=0
    while (i<size_p):
        _=numbers = list(range(n_jobs))
        random.shuffle(_)
        if(_ not in population):
            population.append(_)
            i=i+1
    return population
        
def fitness(schedule,processing_times):
    return compute_makespan(schedule,processing_times)

def crossover(schedule1,schedule2):
    n=len(schedule1)
    i=random.randint(0,n-1)
    j=random.randint(i+1,n)
    new_schedule1=[-1]*n
    new_schedule2=[-1]*n
    
    for idx in range(i,j):
        new_schedule1[idx]=schedule1[idx]
        new_schedule2[idx]=schedule2[idx]
        
    idx=j%n
    for k in range(n):
        if(schedule2[k] not in new_schedule1):
            new_schedule1[idx]=schedule2[k]
            idx=(idx+1)%n
    
    idx=j%n
    for k in range(n):
        if(schedule1[k] not in new_schedule2):
            new_schedule2[idx]=schedule1[k]
            idx=(idx+1)%n
    
    
    return new_schedule1,new_schedule2

def mutation_swap(schedule):
    i=random.randint(0,len(schedule)-2)
    j=random.randint(i+1,len(schedule)-1)
    schedule[i],schedule[j]=schedule[j],schedule[i]
    return schedule

def mutation_reverse_sequence(schedule):
    i=random.randint(0,len(schedule)-2)
    j=random.randint(i+1,len(schedule))
    schedule=schedule[:i]+list(reversed(schedule[i:j]))+schedule[j:]
    return schedule

def roulette_wheel_selection(population, processing_times):
    fitness_values = [fitness(schedule, processing_times) for schedule in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness_value / total_fitness for fitness_value in fitness_values]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    selection_point = random.uniform(0, 1)
    for i, probability in enumerate(cumulative_probabilities):
        if selection_point <= probability:
            return population[i]

def tournament_selection(population,tournament_size,processing_times):
    tournament = random.sample(population, tournament_size)
    winner = min(tournament, key=lambda x: fitness(x, processing_times))
    return winner
    
def elitism(population,n_select,processing_times):
    fitness_values = [(schedule, fitness(schedule, processing_times)) for schedule in population]
    sorted_population = [x[0] for x in sorted(fitness_values, key=lambda x: x[1])]
    return sorted_population[:n_select]

### Main algo

In [7]:
def genetic_algo(processing_times,size_population,num_generations,tournament_size,crossover_rate,mutation_rate,n_select):
    n_jobs,n_machines=processing_times.shape
    population=init_population(n_jobs,size_population)
    
    for _ in range(num_generations):
        #elitism
        hello=elitism(population,n_select,processing_times)
        idx=0
        #crossover
        for i in range(size_population):
            if(random.uniform(0,1)<crossover_rate):
                #selection
                schedule1=tournament_selection(population,tournament_size,processing_times)
                schedule2=tournament_selection(population,tournament_size,processing_times)
                new1,new2=crossover(schedule1,schedule2)
                obj1,obj2=fitness(new1,processing_times),fitness(new2,processing_times)
                if(obj1>obj2):
                    population[i]=new2
                else:
                    population[i]=new1
            elif(idx<n_select):
                population[i]=hello[idx]
                idx=idx+1
                    
        for i in range(size_population):
            if(random.uniform(0,1)<mutation_rate):
                population[i]=mutation_reverse_sequence(population[i])
                
    fitness_values = [(schedule, fitness(schedule, processing_times)) for schedule in population]
    # Sort the population by fitness in ascending order
    fitness_values.sort(key=lambda x: x[1])
    best_one=fitness_values[0]
    return best_one

In [66]:
instance=FmCmax.FmCmax_Instance.read_txt("../TP02-Heuristiques/data/random_instance.txt")
n = instance.n
m = instance.m
M = np.array(instance.P)
schedule=genetic_algo(M,size_population=500,num_generations=100,tournament_size=30,crossover_rate=0.8,mutation_rate=0.8,n_select=10)
print(schedule)

([3, 2, 7, 6, 8, 0, 1, 4, 5, 9], 1102)


In [70]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule,obj=genetic_algo(M,size_population=400,num_generations=100,tournament_size=100,crossover_rate=0.8,mutation_rate=0.8,n_select=10)
    print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.515084621044886 %
instance 3 : 1.572617946345976 %
instance 4 : 1.237432327919567 %
instance 5 : 1.132686084142395 %
instance 6 : 1.2552301255230125 %
instance 7 : 0.9685230024213075 %
instance 8 : 0.0 %
instance 9 : 2.032520325203252 %
instance 10 : 1.263537906137184 %


In [11]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule,obj=genetic_algo(M,size_population=100,num_generations=50,tournament_size=10,crossover_rate=0.8,mutation_rate=0.8,n_select=2)
    print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.5886681383370125 %
instance 3 : 3.145235892691952 %
instance 4 : 3.2482598607888633 %
instance 5 : 3.802588996763754 %
instance 6 : 2.426778242677824 %
instance 7 : 0.9685230024213075 %
instance 8 : 5.638474295190713 %
instance 9 : 7.3983739837398375 %
instance 10 : 2.0758122743682312 %


### ANT COLONY
link (https://www.researchgate.net/profile/Thomas-Stuetzle/publication/2593620_An_Ant_Approach_to_the_Flow_Shop_Problem/links/0046353a2c198330ee000000/An-Ant-Approach-to-the-Flow-Shop-Problem.pdf)

In [11]:
#utilities
def local_search(solution,processing_times):
    for i in range(num_jobs):
        for j in range(num_jobs):
            if i != j:
                new_solution = solution.copy()
                new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
                new_makespan = compute_makespan(new_solution,processing_times)
                if new_makespan < compute_makespan(solution,processing_times):
                    solution = new_solution
    return solution
def insertion_local_search(solution, processing_times): # generated by chatgpt not verified yet
    num_jobs = len(solution)
    solution=list(solution)
    makespan = compute_makespan(solution, processing_times)
    
    # Iterate over each job in the solution
    for i in range(1, num_jobs):
        # Save the job that will be moved
        job = solution[i]
        
        # Try moving the job to every position in the sequence
        for j in range(i):
            # Remove the job from its current position
            solution.pop(i)
            
            # Insert the job at the new position
            solution.insert(j, job)
            
            # Compute the makespan of the new solution
            new_makespan = compute_makespan(solution, processing_times)
            
            # If the new solution is better, accept it
            if new_makespan < makespan:
                makespan = new_makespan
                # Continue searching from the new solution
                i = 1
                break
            else:
                # Revert the solution to its previous state
                solution.pop(j)
                solution.insert(i, job)
    
    return solution, makespan


def construct_sequence(num_jobs, pheromone_matrix):
    # Initialize sequence with a random job
    sequence = [random.randint(0, num_jobs - 1)]
    remaining_jobs = set(range(num_jobs)) - set(sequence)

    # Iterate  remaining jobs
    while remaining_jobs:
        current_job = sequence[-1]
        probabilities = []
        for job in remaining_jobs:
            probability = pheromone_matrix[current_job][job]
            probabilities.append(probability)

        # choose the next job
        probabilities = np.array(probabilities) / np.sum(probabilities)
        next_job = np.random.choice(list(remaining_jobs), p=probabilities)
        sequence.append(next_job)
        remaining_jobs.remove(next_job)

    return sequence


In [25]:
def max_min_ant_system(processing_times,num_ants,rho,q,max_iterations):
    num_jobs,num_machines=processing_times.shape
#     pheromone_trails[i,j,k] represents the amount of pheromone associated with the decision of assigning job i immediately before job j on machine k.
    pheromone_trails = np.ones((num_jobs, num_jobs))

    best_solution = None
    best_makespan = np.inf
    
    pheromone_min = 0.01
    pheromone_max = 1.0
    
    # Loop over iterations
    for iteration in range(max_iterations):
        ant_solutions = []
        ant_makespans = []
        
        #construct a solution for the ant
        #Improve solution by local search
        for ant in range(num_ants):
#             solution = construct_solution(processing_times, pheromone_trails)
#             return solution
            solution=construct_sequence(num_jobs, pheromone_trails)
#             solution=np.random.permutation(num_jobs)
#             return solution
            if(random.uniform(0,1)<0.5):
                solution,_=insertion_local_search(solution, processing_times)
            else:
                solution = local_search(solution,processing_times)
            makespan = compute_makespan(solution,processing_times)
            if makespan < best_makespan:
                best_solution = solution
                best_makespan = makespan
            ant_solutions.append(solution)
            ant_makespans.append(makespan)
        
        
        # Update pheromone trails
        delta_pheromones = np.zeros((num_jobs, num_jobs))
        for ant in range(num_ants):
            for i in range(num_jobs):
                for j in range(num_jobs):
                    if i != j:
                        delta_pheromones[i,j] += q / ant_makespans[ant] if ant_solutions[ant][i] == j else 0
        
        pheromone_trails *= (1 - rho)
        pheromone_trails += delta_pheromones
        
        # Perform MAX-MIN pheromone trail update
        for i in range(num_jobs):
            for j in range(num_jobs):
                if i != j:
                    pheromone_trails[i,j] = max(pheromone_min, min(pheromone_max, pheromone_trails[i,j]))
        pheromone_trails /= np.sum(pheromone_trails)
    
    return best_solution

In [21]:
import numpy as np
import random
# Define FSP problem parameters
instance=FmCmax.FmCmax_Instance.read_txt("../TP02-Heuristiques/data/random_instance.txt")
num_jobs = instance.n
num_machines = instance.m
processing_times = np.array(instance.P)



# Define algorithm parameters
num_ants = 10
alpha = 1
beta = 2
rho = 0.5
q = 100
max_iterations = 100

sol=max_min_ant_system(processing_times,num_ants,rho,q,max_iterations)
sol,compute_makespan(sol,processing_times)

([3, 2, 7, 6, 8, 0, 1, 4, 5, 9], 1102)

In [23]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=max_min_ant_system(M,num_ants=50,rho=0.4,q=10,max_iterations=400)
    obj=compute_makespan(schedule,M)
    print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 1 : 0.4694835680751174 %
instance 2 : 0.07358351729212656 %
instance 3 : 3.2377428307123033 %
instance 4 : 2.2428460943542152 %
instance 5 : 1.132686084142395 %
instance 6 : 1.3389121338912133 %
instance 7 : 0.9685230024213075 %
instance 8 : 1.7412935323383085 %
instance 9 : 2.1951219512195124 %
instance 10 : 2.3465703971119134 %


In [30]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=4
M = np.array(taillard[i]["P"]).transpose()
upper_bound = taillard[i]["ub"]
schedule=max_min_ant_system(M,num_ants=60,rho=0.45,q=5,max_iterations=500)
obj=compute_makespan(schedule,M)
print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 5 : 1.2135922330097086 %


In [13]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=max_min_ant_system(M,num_ants=30,rho=0.4,q=100,max_iterations=300)
    obj=compute_makespan(schedule,M)
    print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 1 : 0.0 %
instance 2 : 0.07358351729212656 %
instance 3 : 4.070305272895467 %
instance 4 : 2.5522041763341066 %
instance 5 : 1.4563106796116505 %
instance 6 : 1.6736401673640167 %
instance 7 : 0.9685230024213075 %
instance 8 : 0.9950248756218906 %
instance 9 : 2.032520325203252 %
instance 10 : 2.0758122743682312 %


In [16]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=max_min_ant_system(M,num_ants=20,rho=0.3,q=500,max_iterations=300)
    obj=compute_makespan(schedule,M)
    print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.29433406916850624 %
instance 3 : 3.330249768732655 %
instance 4 : 2.6295436968290797 %
instance 5 : 1.3754045307443366 %
instance 6 : 1.506276150627615 %
instance 7 : 0.9685230024213075 %
instance 8 : 2.8192371475953566 %
instance 9 : 1.8699186991869918 %
instance 10 : 4.061371841155235 %


In [15]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
for i in range(10):
    M = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    schedule=max_min_ant_system(M,num_ants=50,rho=0.3,q=200,max_iterations=200)
    obj=compute_makespan(schedule,M)
    print("instance",i+1,":",100*(obj-upper_bound)/upper_bound,"%")

instance 1 : 1.486697965571205 %
instance 2 : 0.14716703458425312 %
instance 3 : 3.7927844588344124 %
instance 4 : 3.17092034029389 %
instance 5 : 1.132686084142395 %
instance 6 : 0.6694560669456067 %
instance 7 : 0.9685230024213075 %
instance 8 : 1.9900497512437811 %
instance 9 : 2.1951219512195124 %
instance 10 : 2.1660649819494586 %


#### using insertion local search (recommended by the paper)